<a href="https://colab.research.google.com/github/squadOito/soulcodead2/blob/joseaureliok%2Fnotebook/notebooks/notebook_mongoDB_carregamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Projeto Final**
Escola: SoulCode Academy

Curso: Bootcamp Analista de Dados - Martech - AD2

**Alunos: Nomes, Nomes**

Professores: Douglas Ribeiro e Franciane Rodrigues

## Preparação de Ambiente

In [1]:
# Instalação bibliotecas
!pip install -q pymongo
!pip install -q gcsfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.6/603.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 9.3 MB/s eta 0:00:00


In [2]:
# Importação de bibliotecas
import os
import pandas as pd
import numpy as np

from google.colab import data_table
from google.cloud import storage
from google.colab import drive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from pymongo import MongoClient

### Função Chave

In [3]:
def connectKey (chave):
  '''
  Conecta ao drive para obtenção de chave
  chave = nome da chave em string
  '''
  # Cria compartilhamento com Google Drive
  drive.mount('/content/drive', force_remount=True)

  # Arquivo a ser acessado na pasta compartilhada
  target = chave

  # Caminho completo da pasta compartilhada
  folder = '/content/drive/MyDrive/Classroom/AD2 - Analista de Dados/ProjetoFinal'

  # Acesso ao arquivo no colab
  return os.path.join(folder, target)



### Compartilhamento Chave GDrive

In [4]:
# Nome da chave a ser acessada
chaveGcp = 'projeto-final-ad2-e8-ae566c3a2c2b.json'

# Chave GCP
serviceAccount = connectKey(chaveGcp)

Mounted at /content/drive


In [5]:
# Nome da chave a ser acessada
chaveMongo = 'X509-cert-5608832275999674572.pem'

# Chave MongoDB
mongoKey = connectKey(chaveMongo)

Mounted at /content/drive


### Configuração Conexão MongoDB

In [6]:
# configurando conexão
uri = "mongodb+srv://energy.drstakp.mongodb.net/?authSource=%24external&authMechanism=MONGODB-X509&retryWrites=true&w=majority"
client = MongoClient(uri,
					 tls=True,
					 tlsCertificateKeyFile= mongoKey)

### Configuração da bucket

In [7]:
# Nome da bucket
bucket_name = 'projeto-final-ad2-e8'

# Caminho do prefixo
prefix_name = 'dados/tratados/'

In [8]:
# Conexão com a conta do Google Cloud
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [9]:
# Conexão com a bucket do Google Cloud
client = storage.Client()
bucket = client.get_bucket(bucket_name)

### Conjuntos de dados

In [10]:
# Formatação tabular
data_table.enable_dataframe_formatter()

#data_table.disable_dataframe_formatter()

In [11]:
# Listando conjuntos de dados na bucket
blobList = [x.name for x in bucket.list_blobs(prefix = prefix_name)]

# dataframe lista blobs
blobDf = pd.DataFrame(blobList)

blobDf

,0
0,dados/tratados/
1,dados/tratados/dataset_tratados_xlsx/
2,dados/tratados/dataset_tratados_xlsx/2_1_cap_i...
3,dados/tratados/dataset_tratados_xlsx/2_2_cap_i...
4,dados/tratados/dataset_tratados_xlsx/2_2_cap_i...
5,dados/tratados/dataset_tratados_xlsx/2_3_gerac...
6,dados/tratados/fator_capacidade_mwh_2015_2023.csv
7,dados/tratados/geracao_distribuida_tratado.csv
8,dados/tratados/iea_ponto_recarga_tratado.csv
9,dados/tratados/iea_veiculos_tratado.csv


### Seleção do conjunto de dados

In [ ]:
# Seleção do blob - CSV
blobC1 = 'dados/tratados/fator_capacidade_mwh_2015_2023.csv'
blobC2 = 'dados/tratados/tratado_geracao_distribuida.csv'
blobC3 = 'dados/tratados/iea_ponto_recarga_tratado.csv'
blobC4 = 'dados/tratados/iea_veiculos_tratado.csv'

# Seleção do blob - EXCEL
blobE1 = 'dados/tratados/dataset_tratados_xlsx/2_1_cap_instalada_por_regiao_e_uf_tratado.xlsx'
blobE2 = 	'dados/tratados/dataset_tratados_xlsx/2_2_cap_instalada_de_geracao_eletrica_por_fonte_mw.xlsx'
blobE3 = 	'dados/tratados/dataset_tratados_xlsx/2_3_geracao_eletrica_por_fonte_gwh_tratado.xlsx'

In [ ]:
# Caminho gsutil do arquivo

pathC = {
        'fator_capacidade' : f'gs://{bucket_name}/{blobC1}',
        'geracao_distribuida' : f'gs://{bucket_name}/{blobC2}',
        'iea_recarga' : f'gs://{bucket_name}/{blobC3}',
        'iea_veiculos' : f'gs://{bucket_name}/{blobC4}'
}

pathE = {
        'instalada_regiao' : f'gs://{bucket_name}/{blobE1}',
        'instalada_fonte' : f'gs://{bucket_name}/{blobE2}',
        'geracao_fonte' : f'gs://{bucket_name}/{blobE3}'
}

gs://projeto-final-ad2-e8/dados/tratados/iea_ponto_recarga_bruto.csv


## Collection

In [ ]:
# Criando instância client
db = client['backup_tratados']

# Criando coleções
collection_csv = db['csv']
collection_excel = db['excel']

### Verificação

In [ ]:
# Verificando valores no MongoDB
doc_count_csv = collection_csv.count_documents({})
doc_count_excel = collection_excel.count_documents({})

print( f'\n-CSV: {doc_count_csv} \n-EXCEL: {doc_count_excel}')

## Integração Google Cloud Storage - MongoDB

### Extração da bucket

#### CSV

In [ ]:
dictC = {}

for k, v in pathC:
  dfC = pd.read_csv(v)

  dictC[k] = dfC

print(dictC)

Falha na leitura do DataFrame: gs://projeto-final-ad2-e8/dados/tratados/iea_ponto_recarga_bruto.csv!


#### EXCEL

In [ ]:
dictE = {}

for k, v in pathE:
  dfE = pd.read_excel(v)

  dictE[k] = dfE

print(dictE)

### Carregamento na coleção

In [ ]:
for k, v in dictC:
  df_dict = v.to_dict("records")

  collection_csv.insert_many(df_dict)

In [ ]:
for k, v in dictE:
  df_dict = v.to_dict("records")

  collection_excel.insert_many(df_dict)

#### Verificação

In [ ]:
for k, v in dictC:
  df_dict = v.to_dict("records")

  collection_csv.insert_many(df_dict)

# Status

In [ ]:
print("BACKUP CONCLUÍDO COM SUCESSO!!!")